# Medchem Query Language

In [ ]:
import datamol as dm
import pandas as pd

import medchem


## Query system

Medchem provides a query system that can be used for filtering molecules based on a constructed query made of statement compliant with the provided grammar.

Detailed information about the query system can be seen in the documentation

In [ ]:
# from medchem.query import QueryFilter

# # note that whitespace and newlines are ignored in the query
# query = """
# (
#     HASPROP("tpsa" < 80) AND HASPROP("clogp" < 3) AND ! HASALERT("pains")
# ) 
#     OR 
# (
#     HASPROP("n_heavy_atoms" >= 10) 
#     AND 
#     (
#         HASGROUP("Alcohols") 
#         OR 
#         HASSUBSTRUCTURE("[CX3](=[OX1])O", True, 1)
#     )
# )
# """

# data = dm.freesolv()
# query_filter = QueryFilter(query)
# out = query_filter(data.smiles, n_jobs=-1, progress=True)

# any(out)

# ---



# from typing import List
# import datamol as dm
# from medchem.query.eval import QueryFilter


# query = """HASPROP("tpsa" < 120) AND HASSUBSTRUCTURE("[OH]", True)"""
# chemical_filter = QueryFilter(query, parser="lalr")

# mols = dm.data.cdk2().mol[:10]
# dm.to_image(mols, n_cols=3)

# chemical_filter(mols, n_jobs=1)

# chemical_filter

# queries = [
#             """HASPROP("tpsa" > 120) OR ! HASSUBSTRUCTURE("CO")""",  # A OR ! B
#             """(HASPROP("tpsa" > 120)) OR ! (HASSUBSTRUCTURE("CO"))""",  # (A) OR  ! (B)
#             """(HASPROP("tpsa" > 120) OR ! HASSUBSTRUCTURE("CO"))""",  # (A OR ! B)
#             """HASPROP("tpsa" > 120) OR NOT HASSUBSTRUCTURE("CO")""",  # A OR NOT B
#             """HASPROP("tpsa" > 120) OR ~HASSUBSTRUCTURE("CO")""",  # A OR ~B
#             """HASPROP("tpsa" >   120)   OR ! HASSUBSTRUCTURE("CO")""",  # change in space
#         ]

# from lark import Lark, Transformer, v_args
# from medchem.utils.loader import get_grammar
# from medchem.query.parser import QueryParser

# grammar = Lark(get_grammar(as_string=True), parser="lalr", transformer=QueryParser())

# queries = [
#             # complex query
#             """(HASPROP("tpsa" < 100) AND HASPROP("clogp" < 3) AND ! HASALERT("pains")) OR (HASPROP("n_heavy_atoms" >= 10) AND (HASGROUP("Alcohols") OR HASSUBSTRUCTURE("[CX3](=[OX1])O", True, 1)))""",
#             # is a rewriting of the above
#             """(HASPROP("tpsa" < 100) AND HASPROP("clogp" < 3) AND ! HASALERT("pains")) OR (HASPROP("n_heavy_atoms" >= 10) AND (HASGROUP("Alcohols") OR HASSUBSTRUCTURE("[CX3](=[OX1])O", True, min, 1)))""",
#             # is a rewriting of the above with spacing and a differentm yet equivalent bool expression
#             """
#             (
#                 HASPROP("tpsa" < 100) 
#                 AND 
#                 HASPROP("clogp" < 3) 
#                 AND 
#                 ! HASALERT("pains")
#             ) 
#             OR 
#             (
#                 HASPROP("n_heavy_atoms" >= 10) 
#                 AND 
#                 HASGROUP("Alcohols")
#             ) 
#             OR 
#             (
#                 HASPROP("n_heavy_atoms" >= 10) 
#                 AND 
#                 HASSUBSTRUCTURE("[CX3](=[OX1])O", True)
#             )
#             """,
#             # always true
#             """(HASPROP("tpsa" < 100) AND HASPROP("clogp" < 3)) OR TRUE""",
#             # always false
#             """(HASPROP("tpsa" < 100) AND HASPROP("clogp" < 3)) AND False""",
# ]
# out = [grammar.parse(queries[i]) for i in range(len(queries))]

# out

# [out[i] == out[0] for i in range(len(out))]

# bad_queries = [
#     "(((HASPROP(tpsa > 120 ) | HASSUBSTRUCTURE(c1ccccc1)) AND NOT HASALERT(pains)) OR HASSUBSTRUCTURE([OH]))",
#     """HASPROP("tpsa" > 120 ) OR OR HASSUPERSTRUCTURE("Cc1cc2ccccc2cc1C")""",
#     """HASPROP("tpsa" > 120 ) OR HASSUPERSTRUCTURE("Cc1cc2ccccc2cc1C") OR NOT TRUE""",
#     """HASPROP("tpsa" > 120 ) OR HASSUPERSTRUCTURE("Cc1cc2ccccc2cc1C") OR true""",
#     """HASPROP("tpsa" > 120 ) OR HASSUPERSTRUCTURE("Cc1cc2ccccc2cc1C") TRUE""",  # wrong syntax, UnexpectedCharacters
#     """HASPROP("tpsa" > 120 ) OR  (HASSUPERSTRUCTURE("Cc1cc2ccccc2cc1C")""",
#     """HASPROP("tpsa" > 120 ) OR () OR HASSUPERSTRUCTURE("Cc1cc2ccccc2cc1C")""",
#     """HASPROP("tpsa" > 120) OR HASSUBSTRUCTURE("[OH]", True, >, 3)""",
#     """HASPROP("tpsa" + 120) OR HASSUBSTRUCTURE("[OH]")""",
#     """HASPROP("tpsa", >, 120) OR HASSUBSTRUCTURE("[OH]")""",
#     """HASPROP("tpsa" > 120) OR HASSUBSTRUCTURE("[OH]" True max 3)""",
#     """HASPROP("tpsa" > 120) OR HASSUBSTRUCTURE("[OH]", True, 3)""",
#     """(
#             HASPROP("tpsa" < 100) AND 
#             HASPROP("clogp" < 3) AND 
#             ! HASALERT("pains")
#         ) 
#         OR 
#         (
#             HASPROP("n_heavy_atoms" >= 10) AND 
#             HASGROUP("Alcohols")
#         ) 
#         OR 
#         (
#             HASPROP("n_heavy_atoms" >= 10) AND 
#             HASSUBSTRUCTURE("[CX3](=[OX1])O", True)
#         )""",
#     """(HASPROP("tpsa" < 100) AND HASPROP("clogp" < 3) AND ! HASALERT("pains")) OR (HASPROP("n_heavy_atoms" >= 10) AND (HASGROUP("Alcohols") OR HASSUBSTRUCTURE("[CX3](=[OX1])O", True, min, 1)))""",
# ]

# from medchem.query import QueryOperator
# df = dm.cdk2()
# df["tpsa"] = df["mol"].apply(dm.descriptors.tpsa)
# df["clogp"] = df["mol"].apply(dm.descriptors.clogp)
# df["n_heavy_atoms"] = df["mol"].apply(dm.descriptors.n_heavy_atoms)
# df["has_carboxyl"] = df["mol"].apply(
#     lambda x: QueryOperator.hassubstructure(x, "[CX3](=[OX1])O", True)
# )
# df["has_pains"] = df["mol"].apply(lambda x: QueryOperator.hasalert(x, "pains"))
# df["has_alcohol"] = df["mol"].apply(
#     lambda x: QueryOperator.hasgroup(x, "Alcohols")
# )

# tmp = df.query(
#     "((tpsa < 100) & (clogp < 3) & ~has_pains) | (n_heavy_atoms >= 10 & (has_carboxyl | has_alcohol))"
# )
# df["results"] = False
# df.loc[tmp.index, "results"] = True

# expected_complex_result.index

# mol = dm.to_mol("Oc1cscc1-c1ccc(OC)cc1") # contains tiophene hydroxy should match pains
# substruct1 = dm.to_mol("CO")
# mol.GetSubstructMatches(substruct1, uniquify=True)

# from medchem.rules.rule_filter import RuleFilters

# r = RuleFilters(["rule_of_five", "rule_of_three"])
# r([mol])

# from medchem.groups import _get_functional_group_map

# d = _get_functional_group_map()

# [x for x in d.keys() if 'hydroxy' in x.lower()]